In [1]:
        # citing : Sara Soueidan - Project 3, Tim Book - API Code; Zoom recording DSIR-824 

In [7]:
# imports 
import pandas as pd 
import numpy as np

# APi
import requests

# automating - use for time based operations
import time
import datetime
import sys
from time import sleep

In [9]:
# build a api function,  to automate the process
# define function, with subreddit, # of times function should run 
def get_posts(subreddit, n_iter, epoch_right_now):
    # store base url
    base_url = 'https://api.pushshift.io/reddit/search/submission/?subreddit='
    # create empty list
    df_list = []
    # set epoch to current time
    current_time = epoch_right_now
    #set up for loop
    for post in range(n_iter):
        # get requests
        res = requests.get(
            # base url for response variable
            base_url,
            # parameters for response
            params = {
                # subreddit
                'subreddit' : subreddit,
                # size
                'size'      : 80,
                # lang == True
                'lang'      : True,
                # before this time pull everything
                'before'    : current_time} # close parameters
        ) # close .get
        # take data from most recent request, store as df
        df = pd.DataFrame(res.json()['data'])
        # pull specific columns for df
        df = df[['subreddit', 'selftext', 'title', 'created_utc', 'author', 'permalink']]
        # create submission_title column
#         link_list = list(df['permalink'].str.split('/'))
#         sub_list = []
#         for i in link_list:
#             sub_list.append(i[5])
#         df['submission_title'] = sub_list
#         #drop permalink column
#         df.drop(columns='permalink', inplace=True)
        # append to empty dataframe list
        df_list.append(df)
        # set current time counter back to last epoch in recently appended df
        current_time = df['created_utc'].min()
        #set sleep time
        time.sleep(10)
    #return one df for all requests
    return pd.concat(df_list, axis=0)

In [10]:
# create large df of both subreddits
reddit_df = pd.concat([get_posts('sportsbook', 1, 1616950893), get_posts('dfsports', 1, 1614777568)])

In [12]:
reddit_df.head()

,subreddit,selftext,title,created_utc,author,permalink
0,sportsbook,See title you degenerates,Whats the NCAAB lock today so I can fade this ...,1616949292,Sea_Seaworthiness,/r/sportsbook/comments/mf4bvt/whats_the_ncaab_...
1,sportsbook,See title\n\nI didnt get to watch the fight bu...,Miocic vs Ngannou o1.5 rounds graded as a loss...,1616947311,tj5jr,/r/sportsbook/comments/mf3nu9/miocic_vs_nganno...
2,sportsbook,Do I cash this out and run into the sunset or ...,"4 for 6 on parlay, take the money and run or r...",1616945518,EVH4104,/r/sportsbook/comments/mf328b/4_for_6_on_parla...
3,sportsbook,Head to head arb\nPointsBet Keselowski at +125...,NASCAR ARBs,1616941453,CentArbitrage,/r/sportsbook/comments/mf1opf/nascar_arbs/
4,sportsbook,Got grown adults here crying like they need to...,People who cry after tailing a post that lost ...,1616933815,brokensuper,/r/sportsbook/comments/mf0dgx/people_who_cry_a...


In [19]:
# output to csv
reddit_df.to_csv('Comments/reddit_comments_test.csv')